# Hybrid Keyword Matching - Aho-Corasick Algorithm

This notebook implements a hybrid keyword matching approach for efficient transaction categorization using the Aho-Corasick algorithm for multi-pattern string matching.

## Key Features
- **Dual Matching Strategy**: 
  - Token-based matching for short keywords (< 4 characters)
  - Aho-Corasick automaton for long keywords (>= 4 characters)
- **Efficient Performance**: O(n + z) complexity where n is text length and z is matches
- **Category Scoring**: Aggregates keyword matches weighted by frequency scores
- **Scalable**: Optimized for large datasets with many keywords

## Performance Metrics
- Counts matches against actual categories
- Calculates accuracy of predictions
- Identifies true positives and misclassifications

In [ ]:
# Import required libraries
import pandas as pd  # Data manipulation
import numpy as np   # Numerical computations
import re            # Regular expression processing
from collections import Counter  # Counting keyword matches
import ahocorasick   # Efficient multi-pattern string matching algorithm

In [ ]:
# Load debit transaction data
df = pd.read_excel("debit_txn_v5.xlsx")

In [ ]:
def normalize(text):
    """
    Normalize narration text for consistent matching.
    
    Steps:
    1. Lowercase conversion
    2. Remove special characters
    3. Collapse multiple spaces
    
    Args:
        text (str): Raw narration
        
    Returns:
        str: Normalized text
    """
    text = str(text).lower()
    text = re.sub(r"[^a-z0-9 ]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [ ]:
# Apply normalization to all narrations
df["narr_norm"] = df["Narration"].apply(normalize)

In [ ]:
# Load keyword frequency data with category labels
# This file contains keywords, their categories, and frequency scores
kw_df = pd.read_excel("clean_keyword_frequency_by_category_debit.xlsx")

In [ ]:
# Tokenize normalized narrations by splitting on whitespace
# This creates a list of words for token-based matching
df['tokens'] = (
    df['narr_norm']
    .fillna('')
    .str.lower()
    .str.split()
)

In [ ]:
# Prepare narration strings (remove spaces for substring matching)
df["narr_str"] = (
    df["narr_norm"]
    .str.lower()
    .str.replace(" ", "", regex=False)  # Remove spaces for continuous matching
)

# Prepare keyword strings (remove spaces for matching)
kw_df = kw_df.dropna(subset=["keyword"])  # Remove keywords with missing values

kw_df["keyword_str"] = (
    kw_df["keyword"]
    .astype(str)
    .str.lower()
    .str.replace(" ", "", regex=False)  # Normalize keyword strings
)

In [ ]:
# Extract unique categories from keyword dataframe
categories = kw_df['category'].unique()

In [ ]:
# Square the scores for increased weighting of high-frequency keywords
# This emphasizes more significant keywords in the matching process
kw_df['score_sq'] = kw_df['score'] ** 2

In [ ]:
# Build two separate lookup structures for different keyword lengths
# Strategy: Use different matching approaches based on keyword length

# Short keywords (< 4 chars): Use token-based matching
short_kw_mask = kw_df['keyword'].str.len() < 4
kw_lookup_short = (
    kw_df[short_kw_mask]
    .set_index(['keyword', 'category'])['score_sq']
    .to_dict()
)

# Long keywords (>= 4 chars): Use Aho-Corasick automaton for efficient matching
long_kw_df = kw_df[~short_kw_mask]

def build_long_category_automaton(df_long):
    """
    Build Aho-Corasick automaton for efficient multi-pattern matching.
    
    Args:
        df_long (DataFrame): Keywords with length >= 4
        
    Returns:
        ahocorasick.Automaton: Compiled automaton ready for matching
    """
    automaton = ahocorasick.Automaton()
    for _, row in df_long.iterrows():
        kw = str(row["keyword"]).lower()
        cat = row["category"]
        score_sq = row["score"] ** 2
        # Each keyword maps to (category, score) for later aggregation
        automaton.add_word(kw, (cat, score_sq))
    automaton.make_automaton()  # Compile automaton
    return automaton

long_automaton = build_long_category_automaton(long_kw_df)

In [ ]:
def token_check(tokens):
    """
    Check tokens against keyword list (used for validation).
    
    Args:
        tokens (list): List of word tokens
        
    Returns:
        Counter: Category scores based on token matches
    """
    scores = Counter()
    for t in tokens:
        for cat in categories:
            val = kw_lookup.get((t, cat), 0)
            if val > 0:
                scores[cat] += val
    return scores

In [ ]:
def hybrid_score(row):
    """
    Calculate category scores using hybrid matching approach.
    
    Combines:
    1. Token-based matching for short keywords
    2. Aho-Corasick matching for long keywords
    
    Args:
        row (Series): Transaction row with narration data
        
    Returns:
        Counter: Category scores aggregated from all matches
    """
    scores = Counter()
    narr_norm = row["narr_norm"]
    tokens = row["tokens"]
    
    # 1. Token-based matching for short keywords
    for t in tokens:
        for cat in categories:
            val = kw_lookup_short.get((t, cat), 0)
            if val > 0:
                scores[cat] += val
    
    # 2. Aho-Corasick matching for long keywords
    for _, (cat, score_sq) in long_automaton.iter(narr_norm.lower()):
        scores[cat] += score_sq
        
    return scores

In [ ]:
# Apply hybrid scoring to all transactions
results = df.apply(hybrid_score, axis=1)
scores_df = pd.DataFrame(results.tolist()).fillna(0)
# Take square root to reverse the squaring done earlier (normalize scores)
scores_df = np.sqrt(scores_df)

# Concatenate scores with original dataframe
df = pd.concat([df, scores_df], axis=1)

In [ ]:
# Predict category as the one with highest score
df["predicted_category"] = scores_df.idxmax(axis=1)

In [ ]:
# Calculate accuracy: count matching predictions (case-insensitive)
(df['Category'].str.strip().str.lower() == df['predicted_category'].str.strip().str.lower()).sum()

np.int64(86960)

In [ ]:
# Count total transactions analyzed
df["Category"].count()

np.int64(113484)